<a href="https://colab.research.google.com/github/Romal27/DSGP-24-Retina94/blob/ChatBot-Development/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd

# Loading the uploaded CSV file
intents_df = pd.read_csv('intents.csv')

# Displaying the first few rows to confirm it loaded correctly
print(intents_df.head())


          tag                                   patterns__001  \
0    greeting                                              Hi   
1       about                   What is diabetic retinopathy?   
2    symptoms  What are the symptoms of diabetic retinopathy?   
3      stages    What are the stages of diabetic retinopathy?   
4  prevention         How can I prevent diabetic retinopathy?   

                                   patterns__002  \
0                                          Hello   
1             Tell me about diabetic retinopathy   
2  How do I know if I have diabetic retinopathy?   
3     Explain the stages of diabetic retinopathy   
4               What are the prevention methods?   

                                     patterns__003 patterns__004  \
0                                              Hey  Good morning   
1                     Explain diabetic retinopathy           NaN   
2                 Symptoms of diabetic retinopathy           NaN   
3  How many stages does 

In [7]:
print(intents_df.columns)


Index(['tag', 'patterns__001', 'patterns__002', 'patterns__003',
       'patterns__004', 'patterns__005', 'responses__001', 'responses__002',
       'responses__003'],
      dtype='object')


We need to combine all the patterns__* columns into one column named pattern

In [8]:
pattern_columns = [col for col in intents_df.columns if col.startswith('patterns__')]
intents_df['pattern'] = intents_df[pattern_columns].apply(lambda row: ' '.join(row.dropna()), axis=1)
